In [1]:
import numpy as np

# Construction d'un réseau de Neurones MLP
## Structure des données
On suppose que l'on a $n$ données différentes dans $\mathbb{R}^p$. Ces données sont stockées dans une grande matrice de taille $(p,n)$ dont la $j$-ème colonne est un vecteur de taille $\mathbb{R}^p$ qui représente la $j$-eme donnée.
On note cette matrice $X_j[i]$ avec $1\le i \le p$ et $1\le j \le n$ tel que le vecteur $X_j$ est la $j$-eme donnée d'entrée.
Ainsi l'exemple suivant représente 4 données dans $\mathbb{R}^3$

In [2]:
X =np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
print(X)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


La première opération que l'on veut faire est, étant donné une matrice $A$ de taille $(q,p)$, de trouver la matrice $Y$ de taille $(q,n)$ telle que pour chaque donnée $j$, on ait $Y_j=AX_j$. Dans l'exemple suivant $q=2$.

In [4]:
A=np.array([[1,2,3],[4,5,6]])
print('A=',A)
Y= np.matmul(A,X)
print ('Y=',Y)

A= [[1 2 3]
 [4 5 6]]
Y= [[ 38  44  50  56]
 [ 83  98 113 128]]


Vous devez trouver pour $Y$

`[[ 38  44  50  56]
 [ 83  98 113 128]]`

Maintenant on veut ajouter à $Y$ un vecteur $b \in \mathbb{R}^q$ tel que pour chaque donnée $j$ on ait $Z_j=Y_j+b$. Pour cela on utilise la commande suivante

In [5]:
b=np.array([1,2])
print(np.outer(b,np.ones(5)))

[[1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2.]]


 Vous devez comprendre la commande précédente et vous en servir pour calculer le vecteur $Z$ tel que $Z_j=AX_j+b$ dans l'exemple suivant :

In [6]:
X =np.array([[1,1,3,4],[5,6,7,8],[9,10,11,12]])
A=np.array([[1,5,3],[4,5,6]])
b=np.array([1,3])
Z= np.matmul(A,X) + np.outer(b,np.ones(4))
print(Z)

[[ 54.  62.  72.  81.]
 [ 86.  97. 116. 131.]]


Vous devez trouver pour $Z$

`[[  54.   62.   72.   81.]
 [  86.   97.  116.  131.]]`
 ## Structure d'un réseau de Neurones
 Un réseau de Neurone est une classe notée `Network` qui contient une liste de couches qui sont des objets de la classe `Layer`. On s'intéresse d'abord à construire les couches avant de construire le réseau. La structure globale est la suivante, on la remplira au fur et à mesure du TP.

In [26]:
class Layer() : # Une classe 
    def __init__(self,nb_entree,nb_sortie) :
        self.n_entree = nb_entree
        self.n_sortie= nb_sortie
        self.A=np.random.randn(nb_sortie, nb_entree)
        self.b=np.random.randn(nb_sortie)
    def activation(self,u) :
        return np.arctan(u)
    def forward(self,X) : # calcul de activation(Ax+b)
        Y = np.matmul(self.A, X) + np.outer(self.b, np.ones(X.shape[1]))
        return self.activation(Y)
    def deriv_activation(self,u):
        pass
    def backward(self,X,gx) :  # retropropagation du gradient sur la couche
        pass

class Network() : # Réseau de neurone qui est essentiellement une liste de Layers
    def __init__(self,layers_dim) :
        self.list_layers = []
        for i in range(len(layers_dim) - 1):
            self.list_layers.append(Layer(layers_dim[i], layers_dim[i +1]))
    def forward(self,Z) : # calcul du passage dans chaque couche
        X_list = []
        X_list.append(np.copy(Z))
        for layer in self.list_layers:            
            Z = layer.forward(Z)
            X_list.append(np.copy(Z))
            
        return X_list
    def backward(self,X_list,gx) : # retropropagation globale du gradient
        pass

## Calcul du forward du réseau de Neurone
On va d'abord travailler sur la classe `Layer` et remplir la fonction `__init__()`. La fonction `__init__` prend en argument deux entiers `nb_entree` et `nb_sortie` (notés $p$ et $q$ dans ce qui suit) correspondant à respectivement à la taille des vecteurs d'entrée et la taille des vecteurs de sortie. Ces nombres doivent être stockés dans les variables internes `self.n_entree` et `self.n_sortie`. De plus la fonction `__init__` va tirer de manière aléatoire une matrice $A$ de taille $(q,p)$ (stockée dans `self.A`) et un vecteur $b$ (stocké dans `self.b`) de taille $q$. On utilisera un tirage selon une normale $(0,1)$ avec la fonction `random.randn` pour cela.

In [12]:
# TEST DU RESEAU DE NEURONE
np.random.seed(10)
L=Layer(3,2)
print(L.n_entree)
print(L.n_sortie)
print(L.A)
print(L.b)
# Vous devez trouver
#3
#2
#A=[[ 1.3315865   0.71527897 -1.54540029]
#[-0.00838385  0.62133597 -0.72008556]]
#b=[ 0.26551159  0.10854853 ]

3
2
[[ 1.3315865   0.71527897 -1.54540029]
 [-0.00838385  0.62133597 -0.72008556]]
[0.26551159 0.10854853]


On remplit maintenant la fonction `forward` de la classe `Layer`. Etant donné une matrice $X$ de taille $(p,n)$ (où $n$ représente le nombre de données), la fonction `forward` calcule $Y$ de taille $(q,n)$ telle que $Y_j=AX_j+b$ pour tout $j$ entre $1$ et $n$ (on rappelle que $n$ est donné par `X.shape[1]`). On utilisera la section précédente pour faire ce calcul, on fera notamment attention à ne pas utiliser le vecteur $b$ directement dans la somme. Une fois que $Y$ est calculé, la fonction forward doit rendre `self.activation(Y)` (qui est ici déjà codée est qui est une arctangente).

In [16]:
np.random.seed(10)
L = Layer(3,2)
X =np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
print(L.forward(X))
# Vous devez trouver
#[[-1.45681202 -1.44993537 -1.44218059 -1.43336915]
# [-1.2743528  -1.28322907 -1.29160274 -1.29951425]]

[[-1.45681202 -1.44993537 -1.44218059 -1.43336915]
 [-1.2743528  -1.28322907 -1.29160274 -1.29951425]]


 Nous allons maintenant nous intéresser à la construction du réseau de Neurone. La classe Network prend en argument d'entrée `layers_dim` qui est une liste d'entier et qui correspond à la liste des dimensions des différentes couches. Si par exemple `layers_dim` vaut `[3,5,7,6]` alors le réseau de Neurone va prendre en argument des vecteurs de taille `layers_dim[0]=3` et va rendre au final des vecteurs de taille `layers_dim[-1]=6`. Au final le réseau de neurones sera composé de `3` couches, une de taille `(3,5)`, puis une de taille `(5,7)` puis une de taille `(7,6)`. Modifiez la fonction `__init__` de Network pour que le réseau construire ces couches et stocke les couches au fur et à mesure dans la liste `self.list_layers`. Puis lancez le code de vérification suivant :

In [20]:
N=Network([1,5,2,6])
for i,layer in enumerate(N.list_layers) :
    print('la couche '+str(i)+ ' prend des vecteurs de taille ' + str(layer.n_entree)+' et sort des vecteurs de taille '+ str(layer.n_sortie))   
# Vous devez trouver :
#la couche 0 prend des vecteurs de taille 1 et sort des vecteurs de taille 5
#la couche 1 prend des vecteurs de taille 5 et sort des vecteurs de taille 2
#la couche 2 prend des vecteurs de taille 2 et sort des vecteurs de taille 6

la couche 0 prend des vecteurs de taille 1 et sort des vecteurs de taille 5
la couche 1 prend des vecteurs de taille 5 et sort des vecteurs de taille 2
la couche 2 prend des vecteurs de taille 2 et sort des vecteurs de taille 6


Finalement on va programmer le réseau de Neurone en entier !! Il s'agit de la fonction forward de la classe Network, c'est très simple, on prend en argument une matrice `Z` et on la fait passer successivement dans toutes les couches du réseau. On pensera cependant à copier à chaque passage de couche le résultat dans une liste notée `X_list`, on en aura besoin pour plus tard. Attention aussi quand vous copiez un vecteur , utilisez la fonction `np.copy` sinon le vecteur n'est pas vraiment copié. On pensera aussi à copier le vecteur `Z` et à le mettre en tout début de `X_list`. Ainsi `X_list` est une suite de matrices dont la première dimension est exactement donnés par la variable `layers_dim`. La fonction `forward` devra rendre `X_list`. Le résultat du réseau de neurone est exactement donné par `X_list[-1]`

In [27]:
np.random.seed(10)
N=Network([3,5,6,7])
X=np.arange(300).reshape(3,100)
X_list=N.forward(X)
for X in X_list :
    print(X.shape)
# Vous devez trouver
#(3, 100)
#(5, 100)
#(6, 100)
#(7, 100)
print(np.linalg.norm(X_list[-1]))
# vous devez trouver
# 28.280716503

(3, 100)
(5, 100)
(6, 100)
(7, 100)
28.28071650303861


## Calcul du gradient du réseau de Neurone
On va calculer maintenant calculer la rétropropagation du gradient du réseau de neurone. Un conseil, commencez par copier votre classe ici pour ne plus toucher à la classe de la section précédente

In [123]:
class Layer() : # Une classe 
    def __init__(self,nb_entree,nb_sortie) :
        self.n_entree = nb_entree
        self.n_sortie= nb_sortie
        self.A=np.random.randn(nb_sortie, nb_entree)
        self.b=np.random.randn(nb_sortie)
    def activation(self,u) :
        return np.arctan(u)
    def forward(self,X) : # calcul de activation(Ax+b)
        Y = np.matmul(self.A, X) + np.outer(self.b, np.ones(X.shape[1]))
        return self.activation(Y)
    def deriv_activation(self,u):
        return 1 / (1 + (u)**2)
    def backward(self,X,gx) :  # retropropagation du gradient sur la couche
        temp1 = np.matmul(self.A, X)
        temp2 = self.deriv_activation(temp1 + np.outer(self.b, np.ones(X.shape[1])))
        M = np.multiply(temp2, gx)  
        ge = np.matmul(np.matrix.transpose(self.A), M)
        ga = np.matmul(M,np.matrix.transpose(X))
        gb = np.zeros(self.n_sortie)
        for i in range(self.n_sortie):
            gb[i] = np.sum(M[i])
        return  ge, ga, gb

class Network() : # Réseau de neurone qui est essentiellement une liste de Layers
    def __init__(self,layers_dim) :
        self.list_layers = []
        for i in range(len(layers_dim) - 1):
            self.list_layers.append(Layer(layers_dim[i], layers_dim[i +1]))
    def forward(self,Z) : # calcul du passage dans chaque couche
        X_list = []
        X_list.append(np.copy(Z))
        for layer in self.list_layers:            
            Z = layer.forward(Z)
            X_list.append(np.copy(Z))
            
        return X_list
    def backward(self,X_list,gx) : # retropropagation globale du gradient
        list_grad = []
        
        for (layer, i) in zip(reversed(self.list_layers), reversed(range(len(X_list)))):
            (gx, ga, gb) = layer.backward(X_list[i-1], gx)
            list_grad.append((ga, gb))
        return list(reversed(list_grad))

Tout d'abord on s'occupe de la classe Layer, et on remplit la fonction `deriv_activation`. C'est simple, si on note $\phi$, la fonction `activation`, alors on met la dérivée $\phi'$ dans la fonction `deriv_activation`. A vous de jouer..


In [34]:
l=Layer(10,5)
X=np.arange(3)+0.56
print(l.activation(X))
print(l.deriv_activation(X))
# Vous devez trouver
#[ 0.51048832  1.00075586  1.19839788]
#[ 0.76126675  0.29123952  0.13238721]

[0.51048832 1.00075586 1.19839788]
[0.76126675 0.29123952 0.13238721]


C'était simple... Maintenant il faut coder la rétropropagation pour une couche. On rappelle que si la couche est de taille $(p,q)$ avec $n$ données alors l'algorithme prend en entrée `gx` un vecteur de taille $(q,n)$, calcule le gradient par rapport à $A$ et $b$ (qui sont donc des vecteurs de taille $(q,p)$ et $q$ respectivement) et rend un vecteur de taille $(p,n)$ qui sert à rétropropager le gradient aux couches précédentes. Les formules de calcul sont :

$$ M=\phi'(AX+b)*g_x $$
$$g_e = A^T M$$
$$g_A = M X^T$$
$$g_b[i] = \sum_j M_{ij}$$

Ici $g_x$ est le vecteur de taille $(q,n)$ qui est donné en variable, $X$ est de taille $(p,n)$ et sont les données que la couche vient de traiter dans le calcul du forward (ces données sont stockées dans `X_list`), $M$ est un vecteur intermédiaire de taille $(q,n)$ (le produit donné dans la formule est donc entendu comme un produit terme à terme), ensuite $g_e$,$g_A$ et $g_b$ sont les trois gradients calculés (et rendus par l'algorithme). $g_e$ est le gradient utilisé dans la couche du dessous, $g_A$ et $g_b$ sont les gradients par rapport aux variables $self.A$ et $self.b$. Implémentez la fonction backward de Layer qui vous fait ces calculs et vous rend le triplet $g_e,g_A,g_b$ dans cet ordre.
Testez votre code ci-dessous :

In [115]:
np.random.seed(42)
l=Layer(3,2)
X=np.arange(30).reshape(3,10)
GX=np.arange(20).reshape(2,10)
(un,deux,trois)=l.backward(X,GX)
for c in (un,deux,trois) :
    print(c.shape)
    # Vous devez trouver
    #(3,10)
    #(2,3)
    #(2,)
print(un)
print(deux)
print(trois)
#Vous devez trouver
#[[  0.37935363   0.59948105   1.00846542   1.8798579    4.14461384
#   11.62441892  24.25226532  11.42130188   4.7706869    2.53182615]
# [ -0.05832252  -0.09247264  -0.15557944  -0.28972037  -0.63803699
#   -1.78809274  -3.72959337  -1.75699167  -0.73455346  -0.39037505]
# [ -0.05831843  -0.08856387  -0.14876103  -0.28071541  -0.62737057
#   -1.7761148   -3.71655741  -1.74340955  -0.72054256  -0.37603143]]
#[[  7.09467901e-01   1.93955318e+00   3.16963847e+00]
# [  2.42292537e+02   6.52994735e+02   1.06369693e+03]]
#[  0.12300853  41.07021978]

(3, 10)
(2, 3)
(2,)
[[ 0.37935363  0.59948105  1.00846542  1.8798579   4.14461384 11.62441892
  24.25226532 11.42130188  4.7706869   2.53182615]
 [-0.05832252 -0.09247264 -0.15557944 -0.28972037 -0.63803699 -1.78809274
  -3.72959337 -1.75699167 -0.73455346 -0.39037505]
 [-0.05831843 -0.08856387 -0.14876103 -0.28071541 -0.62737057 -1.7761148
  -3.71655741 -1.74340955 -0.72054256 -0.37603143]]
[[7.09467901e-01 1.93955318e+00 3.16963847e+00]
 [2.42292537e+02 6.52994735e+02 1.06369693e+03]]
[ 0.12300853 41.07021978]


Le plus dur est fait... Il reste à coder le backward du réseau en entier. Pour cela on code maintenant la fonction backward de la classe Network. Le principe est simple, on prend en entrée un vecteur de taille $(q,n)$ où $q$ est la taille de la dernière couche et on calcule le gradient pour chaque couche en parcourant les couches en ordre inverse et en prenant pour la couche $s$ le gradient donné par la variable $g_e$ de la couche $s+1$...  On stocke la suite de variables $g_A$ et $g_b$ dans une liste que l'on rend. Si on appelle `list_grad` cette liste de gradient, ce doit être une liste de la taille du nombre de couches dont chaque élément contient un couple qui correspond au calcul de $(g_A,g_b)$ pour la couche en cours. Comme la liste 'list_grad' est construite à l'envers, on prendra bien soin de l'inverser avant de rendre le résultat.
On aura besoin des fonctions suivantes sur les listes :

In [63]:
a=['alice','bob','charles']
b=['toto','titi','tata']
print("# parcours classique d'une liste")
for e in a :
    print(e)
print("# fonction zip qui permet de parcourir 2 listes en même temps")
for (e,f) in zip(a,b) :
    print(e,f)
print("# fonction reversed qui permet de parcourir une liste à l'envers")
for e in reversed(a) :
    print(e) 
print("# fonction qui permet d'inverser une liste")
c=list(reversed(a))
print(c)

# parcours classique d'une liste
alice
bob
charles
# fonction zip qui permet de parcourir 2 listes en même temps
alice toto
bob titi
charles tata
# fonction reversed qui permet de parcourir une liste à l'envers
charles
bob
alice
# fonction qui permet d'inverser une liste
['charles', 'bob', 'alice']


In [124]:
# Vous pouvez tester votre réseau de Neurone avec le code ci-dessous
np.random.seed(42)
N=Network([3,5,6,7])
X0=np.arange(300).reshape(3,100)
gX=np.arange(700).reshape(7,100)
X_list=N.forward(X0)
l=N.backward(X_list,gX)
for (g_a,g_b) in l :
    print (g_a.shape,g_b.shape)
#Vous devez trouver
#((5, 3), (5,))
#((6, 5), (6,))
#((7, 6), (7,))
print("########################")

for (g_a,g_b) in l :
    
    print (np.linalg.norm(g_a),np.linalg.norm(g_b))
#Vous devez trouver
#(245191.63608985924, 1020.2244124421867)
#(19866.562231316475, 6160.3245197739589)
#(95230.475542315864, 35836.812915082708)

(5, 3) (5,)
(6, 5) (6,)
(7, 6) (7,)
########################
245191.63608985924 1020.2244124421866
19866.56223131648 6160.324519773959
95230.47554231588 35836.81291508271


## Interfacage
Ouf, votre reseau de neurones est terminé et opérationnel... On va maintenant ajouter des fonctions d'interfacages avec les autres programmes. Les autres programmes ne voient pas les variables du réseau de neurones comme une liste de matrice et de vecteurs mais comme un grand vecteur, il faut des fonctions pour faire l'interface, elles sont ci-dessous :

In [113]:
def get(N) :
    # prend un réseau de neurone et récupère les variables A et b de chaque couche dans un grand vecteur
    n=0
    #calcul de la taille du vecteur de sortie
    for l in N.list_layers :
        n+=(l.n_entree+1)*l.n_sortie
    result=np.zeros(n) # on construit un vecteur de la bonne taille et on rend le réseau de Neurone
    ind=0 # indice auquel on commence à construire le tableau
    for l in N.list_layers :
        result[ind:ind+l.n_entree*l.n_sortie]=l.A.flatten()
        ind+=l.n_entree*l.n_sortie
        result[ind:ind+l.n_sortie]=l.b.flatten()
        ind+=l.n_sortie   
    return result
def set(N,C) :
    # prend un reseau de neurone et met les variables A et b aux valeurs correspondantes de C
    ind=0
    for l in N.list_layers :
        l.A=C[ind:ind+l.n_entree*l.n_sortie].reshape(l.n_sortie,l.n_entree)
        ind+=l.n_entree*l.n_sortie
        l.b[:]=C[ind:ind+l.n_sortie]
        ind+=l.n_sortie
def recup_grad(N,list_grad) :
    # prend un réseau de neurone et la liste des gradients et la met dans un grand vecteur
    n=0
    #calcul de la taille du vecteur de sortie
    for l in N.list_layers :
        n+=(l.n_entree+1)*l.n_sortie
    result=np.zeros(n) # on construit un vecteur de la bonne taille et on rend le réseau de Neurone
    ind=0 # indice auquel on commence à construire le tableau
    for l,(gA,gB) in zip(N.list_layers,list_grad) :
        result[ind:ind+l.n_entree*l.n_sortie]=gA.flatten()
        ind+=l.n_entree*l.n_sortie
        result[ind:ind+l.n_sortie]=gB.flatten()
        ind+=l.n_sortie   
    return result

In [114]:
np.random.seed(42)
# On crée un réseau de neurone et on le lance sur des données
N=Network([3,5,6,7])
X=np.arange(6).reshape(3,2)
X_list=N.forward(X)
Y=np.copy(X_list[-1])


# On récupère les paramètres et on les met à 0
C=get(N)
set(N,np.zeros(C.shape[0]))
X_list=N.forward(X)
Z=np.copy(X_list[-1])
D=get(N)
print(np.linalg.norm(D), np.linalg.norm(Z)) # on vérifie que tout est nul

#on met les anciens paramètres et on vérifie que on obtient le bon résultat
set(N,C)
X_list=N.forward(X)
Z=np.copy(X_list[-1])
print(np.linalg.norm(Z-Y))

# Test de la fonction recup_grad
gX=np.arange(14).reshape(7,2)
grad=recup_grad(N,N.backward(X_list,gX))
print(grad.shape)


0.0 0.0
0.0
(6, 2)
(7, 6)
(105,)


Si vous avez le temps et l'énergie vous pouvez incorporer comme il vous semble les fonctions d'interfacages directement dans la classe Network. Pensez à ne pas modifier votre classe actuelle mais bien à travailler sur une copie dans l'espace ci-dessous.